In [1]:
import os
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models,Sequential
import cv2, numpy as np
import os

## We are using pretrained weights of inceptionV3
### [from->https://www.kaggle.com/ekaterinadranitsyna/keras-applications-models](https://www.kaggle.com/ekaterinadranitsyna/keras-applications-models)
adding it to our dataset,

In [2]:
train_data_csv = '../input/petfinder-pawpularity-score/train.csv'
test_data_csv  = '../input/petfinder-pawpularity-score/test.csv'
train_folder = '../input/petfinder-pawpularity-score/train'
test_folder  = '../input/petfinder-pawpularity-score/test'

## Hyperparameter for our newtowk

In [3]:
y = 'Pawpularity'
validation_size = 0.25
autotune = tf.data.experimental.AUTOTUNE
img_size = 299
batch_size = 8
dropout = 0.2
lr = 1e-3
decay_steps = 100
decay_rate = 0.96
epochs = 1000

In [4]:
def id_to_path(img_id,dir):
    return os.path.join(dir, f'{img_id}.jpg')

In [5]:
def get_image(path):
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    image = tf.cast(tf.image.resize_with_pad(image, img_size, img_size), dtype=tf.float32)
    return tf.keras.applications.inception_v3.preprocess_input(image)

In [6]:
def process_dataset(path, label):
    return get_image(path), label

In [7]:
def get_dataset(x, y=None):
    if y is not None:
        ds = tf.data.Dataset.from_tensor_slices((x, y))
        return ds.map(process_dataset, num_parallel_calls=autotune) \
            .batch(batch_size).prefetch(buffer_size=autotune)
    else:
        ds = tf.data.Dataset.from_tensor_slices(x)
        return ds.map(get_image, num_parallel_calls=autotune) \
            .batch(batch_size).prefetch(buffer_size=autotune)

In [8]:
data_train = pd.read_csv(train_data_csv)
data_train.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72


In [9]:
data_test = pd.read_csv(test_data_csv)
data_test.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0


In [10]:
data_train['path'] = data_train['Id'].apply(lambda x: id_to_path(x, train_folder))
data_test['path'] = data_test['Id'].apply(lambda x: id_to_path(x, test_folder))

train_subset, valid_subset = train_test_split(
    data_train[['path', y]],
    test_size=validation_size, shuffle=True, random_state=5
)

In [11]:
data_train.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,path
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...


In [12]:
data_train.path

0       ../input/petfinder-pawpularity-score/train/000...
1       ../input/petfinder-pawpularity-score/train/000...
2       ../input/petfinder-pawpularity-score/train/001...
3       ../input/petfinder-pawpularity-score/train/001...
4       ../input/petfinder-pawpularity-score/train/001...
                              ...                        
9907    ../input/petfinder-pawpularity-score/train/ffb...
9908    ../input/petfinder-pawpularity-score/train/ffc...
9909    ../input/petfinder-pawpularity-score/train/ffd...
9910    ../input/petfinder-pawpularity-score/train/fff...
9911    ../input/petfinder-pawpularity-score/train/fff...
Name: path, Length: 9912, dtype: object

In [13]:
train_data = get_dataset(x=train_subset['path'], y=train_subset[y]/100.).shuffle(1000)
valid_data = get_dataset(x=valid_subset['path'], y=valid_subset[y]/100.).shuffle(1000)
test_data = get_dataset(x=data_test['path'])

2021-11-29 17:44:16.210558: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-29 17:44:16.331985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-29 17:44:16.332893: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-29 17:44:16.341420: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [14]:
model = tf.keras.models.load_model('../input/keras-applications-models/InceptionV3.h5')
model.trainable = False

In [15]:
model= tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(shape=(img_size, img_size, 3)),
        tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical"),
        model,
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout, name='top_dropout'),
        tf.keras.layers.Dense(32, activation='elu'),
        tf.keras.layers.Dense(1)
    ]
)

In [16]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=lr,
    decay_steps=decay_steps, decay_rate=decay_rate,staircase=True)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)

In [17]:
from tensorflow.keras import backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true*100 - y_pred*100)))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                    loss=rmse,
                    metrics=[rmse])

In [18]:
history=model.fit(train_data,validation_data=valid_data, epochs=epochs,verbose=1,callbacks=[early_stop],use_multiprocessing=True, workers=-1)

Epoch 1/1000


2021-11-29 17:44:27.568319: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-29 17:44:38.210677: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 219 of 1000
2021-11-29 17:44:48.240488: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 438 of 1000
2021-11-29 17:44:58.221560: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 635 of 1000
2021-11-29 17:45:08.225579: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 883 of 1000
2021-11-29 17:45:10.043956: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.
2021-11-29 17:45:10.769791: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


930/930 [==============================] - ETA: 0s - loss: 53.3326 - rmse: 53.3250

2021-11-29 17:45:55.992752: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 242 of 1000
2021-11-29 17:45:58.610646: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


930/930 [==============================] - 102s 54ms/step - loss: 53.3326 - rmse: 53.3250 - val_loss: 20.0326 - val_rmse: 20.0318
Epoch 2/1000


...

930/930 [==============================] - 76s 48ms/step - loss: 16.5580 - rmse: 16.5466 - val_loss: 18.2499 - val_rmse: 18.2487
Epoch 10/1000


2021-11-29 17:57:55.358760: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 312 of 1000
2021-11-29 17:58:05.356822: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 674 of 1000


  5/930 [..............................] - ETA: 29s - loss: 16.4911 - rmse: 16.4911

2021-11-29 17:58:12.480049: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


930/930 [==============================] - 71s 48ms/step - loss: 16.4505 - rmse: 16.4408 - val_loss: 18.2147 - val_rmse: 18.2132


In [19]:
data_test[y] = model.predict(test_data,use_multiprocessing=True, workers=os.cpu_count())*100

In [20]:
data_test[['Id', y]].to_csv('submission.csv', index=False)
data_test[['Id', y]].head()

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,41.840374
1,43a2262d7738e3d420d453815151079e,44.162552
2,4e429cead1848a298432a0acad014c9d,45.983620
3,80bc3ccafcc51b66303c2c263aa38486,45.296276
4,8f49844c382931444e68dffbe20228f4,45.647762
